In [1]:
import pyspark as ps
import pandas as pd
import boto3

In [9]:
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import udf
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.ml.feature import NGram,Tokenizer
from pyspark.sql.functions import col

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
s3 = boto3.client('s3')
bucket = "blossom-data-engs"

s3.download_file(bucket,"companies.csv",'companies.csv')
s3.download_file(bucket,"alldata.csv",'alldata.csv')

In [5]:
companies = spark.read.csv('companies.csv',header=True, inferSchema=True)
alldata = spark.read.csv('alldata.csv',header=True, inferSchema=True)

In [6]:
alldata = alldata.withColumnRenamed('description','descriptions')
companies = companies.withColumnRenamed('company name','company_name')

In [11]:
alldata.columns

['position', 'company', 'descriptions', 'reviews', 'location']

In [8]:
companies.columns

['ticker',
 'company_name',
 'short name',
 'industry',
 'description',
 'website',
 'logo',
 'ceo',
 'exchange',
 'market cap',
 'sector',
 'tag 1',
 'tag 2',
 'tag 3']

In [171]:
alldata = alldata.alias('alldata')
companies = companies.alias('companies')

In [31]:
ds =  companies.join(alldata, companies.company_name == alldata.company, 'inner')

In [36]:
num = input('Press 1 for unigram and 2 for bigram   ')

Press 1 for unigram and 2 for bigram   2


In [37]:
def ngram(num,ds):
    tokens = Tokenizer(inputCol = 'descriptions', outputCol = 'tokens')
    dt = tokens.transform(ds) 
    ngrams = NGram(n=num, inputCol = 'tokens', outputCol = 'ngrams')
    df = ngrams.transform(dt)
    return df

In [38]:
def outputs(df):
    industry = df.select(['ngrams', 'industry']).limit(3)\
        .select('industry', F.explode('ngrams').alias('ngrams')).groupby(['ngrams','industry']).count()
    city = df.select(['ngrams', 'location']).limit(3)\
        .select('location', F.explode('ngrams').alias('ngrams')).groupby(['ngrams','location']).count()
    city = city.withColumnRenamed('location','city')
    return industry,city

NameError: name 'industry' is not defined